### Accesing to Ollama API

#### Import Libraries

In [12]:
import os
import json
import requests
import dotenv
import ollama

In [ ]:
# en cmd ollama serve

### Setting Up API

In [13]:
OLLAMA_HOST = os.getenv("OLLAMA_HOST", "http://localhost:11434")
MODEL = os.getenv("OLLAMA_MODEL", "deepseek-r1:8b")  # or "llama3.2"

### Generating Text

In [14]:
def _chat_ollama(messages, temperature=None, max_tokens=None):
    """
    Minimal wrapper for Ollama /api/chat.
    messages: list of {"role": "system"|"user"|"assistant", "content": str}
    """
    url = f"{OLLAMA_HOST}/api/chat"
    payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False,
        # map options similar to OpenAI
        "options": {}
    }
    if temperature is not None:
        payload["options"]["temperature"] = float(temperature)
    if max_tokens is not None:
        payload["options"]["num_predict"] = int(max_tokens)

    r = requests.post(url, json=payload, timeout=120)
    r.raise_for_status()
    data = r.json()
    # Ollama returns the last message in 'message'
    return data["message"]["content"].strip()


In [15]:
def ask_llama32(prompt, max_tokens=50, temperature=0.3):
    messages = [
        {"role": "system", "content": "You are a concise and precise assistant."},
        {"role": "user", "content": prompt}
    ]
    return _chat_ollama(messages, temperature=temperature, max_tokens=max_tokens)

In [16]:
prompt = "How are you today?"

In [17]:
generated_text = ask_llama32(prompt, max_tokens=50, temperature=0.7) 
print(prompt, generated_text)


How are you today? <think>
I'm an AI designed to be direct, so my response should match that style while being polite. The user asked "How are you today?"—they're greeting me with interest in my state.

Since I don't have feelings, the


### Summarising Text

In [18]:
def text_summarizer_llama32(prompt, temperature=0.5, max_tokens=256):
    messages = [
        {"role": "system",
         "content": "Extract a comma-separated list of keywords from the provided text."},
        {"role": "user",
         "content": "A flying saucer ... (same example as above)"},
        {"role": "assistant",
         "content": "flying saucer, guest house, 7ft alien-like figure, hedge, cigar-shaped UFO, school yard, extraterrestrial encounters, UK, mass sightings, Broad Haven, Bermuda Triangle, strange beings, late seventies, Netflix documentary, Steven Spielberg, 1977, Cold War, Star Wars, Close Encounters of the Third Kind"},
        {"role": "user",
         "content": "Each April ... (same second example)"},
        {"role": "assistant",
         "content": "April, Maeliya, northwest Sri Lanka, banyan tree, wewa, reservoir, tank, Sinhala, rice paddies, 175-acres, rainwater, agrarian committee, coconut milk, blessings, prosperous harvest, deities, sluice gate, irrigation canals, dry months, lake-like water bodies, farmers, Sinhala phrase, village life, pagoda, temple"},
        {"role": "user", "content": prompt}
    ]
    return _chat_ollama(messages, temperature=temperature, max_tokens=max_tokens)


In [19]:
prompt = "Master Reef Guide Kirsty Whitman didn't need to tell me twice. Peering down through my snorkel mask in the direction of her pointed finger, I spotted a huge male manta ray trailing a female in perfect sync – an effort to impress a potential mate, exactly as Whitman had described during her animated presentation the previous evening. Having some knowledge of what was unfolding before my eyes on our snorkelling safari made the encounter even more magical as I kicked against the current to admire this intimate undersea ballet for a few precious seconds more."
print(prompt)



Master Reef Guide Kirsty Whitman didn't need to tell me twice. Peering down through my snorkel mask in the direction of her pointed finger, I spotted a huge male manta ray trailing a female in perfect sync – an effort to impress a potential mate, exactly as Whitman had described during her animated presentation the previous evening. Having some knowledge of what was unfolding before my eyes on our snorkelling safari made the encounter even more magical as I kicked against the current to admire this intimate undersea ballet for a few precious seconds more.


In [20]:
text_summarizer_llama32(prompt, max_tokens=100)

'<think>\nOkay, let\'s tackle this query. The user provided a text about a snorkeling experience with manta rays and mentions Kirsty Whitman from "Master Reef Guide." They want keywords extracted in CSV format.\n\nFirst, I need to parse the sentence carefully. Key elements here are: Master Reef Guide, Kirsty Whitman, snorkel mask, male manta ray, female manta ray, perfect sync, effort to impress a potential mate, and the previous evening\'s presentation. Also'

### Coding a simple chatbot

In [21]:


def poetic_chatbot_llama32(prompt, temperature=1.0, max_tokens=256):
    messages = [
        {"role": "system", "content": "You are a poetic chatbot. Answer in short rhymed couplets."},
        {"role": "user", "content": "When was Google founded?"},
        {"role": "assistant", "content": "In nineteen ninety-eight the vision came to be,\nBy Page and Brin, a search light for all to see."},
        {"role": "user", "content": "Which country has the youngest president?"},
        {"role": "assistant", "content": "Austria crowned its youth with Kurz’s earnest run,\nAt thirty-one he led, beneath a cautious sun."},
        {"role": "user", "content": prompt}
    ]
    return _chat_ollama(messages, temperature=temperature, max_tokens=max_tokens)

In [22]:
prompt = "When was cheese first made?"
poetic_chatbot_llama32(prompt, temperature=1.0, max_tokens=256)

"<think>\nOkay, let me start by understanding what the user is asking. They want to know when cheese was first made. Hmm, that's an interesting historical question. First, I need to recall the earliest evidence of cheese-making dates back around 5000-6000 BCE in places like Mesopotamia or northern Europe.\n\nBut wait, maybe they're looking for a more specific answer? Different sources might have varying timelines. They could also be interested in different types of cheese or regional variations. Should I mention the regions where it's commonly believed to have started?\n\nThe user's identity isn't clear—could be a student, food enthusiast, or someone curious about history. Their actual need is straightforward: the date of the first cheese-making. But deeper down, they might want context on how cheese evolved or its cultural significance.\n\nI should also consider if they expect an exact year, but ancient foods don't have precise dates usually. Need to present it as an approximation wit

#### Langchain